In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/DATA/데이터분석캡스톤디자인/'

import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Mounted at /content/drive


#Intra-list Similarity
- 추천 시스템이 '서로 비슷한 성격의 메뉴'를 추천해주고 있는 것이 맞는지 평가 
- 3차 필터링까지 마친 뒤 적용
- 3차 필터링에서 사용자가 추천받은 최종 10개의 메뉴 리스트 데이터 필요

1. 각 경우에 대한 intra-list-similarity 구하기

In [ ]:
# user_rec_data = (한 사용자의) 최종 추천 메뉴의 재료 data frame
# 221124_1104 : 지금 코드로는
# recommended = get_recommendations()
# user_rec_data = pd.DataFrame(recommended['요리재료'])

def intra_list_sim(user_rec_data) :

  user_documents = []
  for i in range(10) : # 메뉴 10개
    user_documents.append(user_rec_data.iat[i,0]) # data frame -> list
  
  tf_idf_model = TfidfVectorizer().fit(user_documents) # tf-idf 계산

  ingre_id_list = sorted(tf_idf_model.vocabulary_.items(), key=lambda x:x[1], reverse=False)
  ingre_list = [x[0] for x in ingre_id_list]

  tf_idf_df = pd.DataFrame(tf_idf_model.transform(user_documents).toarray(),columns=ingre_list)

  cos_sim_df = pd.DataFrame(cosine_similarity(tf_idf_df, tf_idf_df))

  total = 0
  for j in range(10) :
    col = cos_sim_df.iloc[j]
    total += sum(col)

  user_sim_total = (total - 10)/2
  user_sim_mean = user_sim_total / 45

  return user_sim_mean

# 여기서 나온 값을 전체 평균 구하기! -> 이게 이 모델의 intra-list-similarity 값

*2*. 각 경우에 대한 intra-list-similarity 평균 구하기
- 기준 1) 사용자가 선택하는 대표 메뉴의 수와 모델의 추천 정확도의 관계
- 기준 2) 사용자가 입력하는 재료 수와 모델의 추천 정확도의 관계
- 기준 3) '2차 필터링'을 단순 찾기로 했을 때와 텍스트 유사도를 적용했을 때의 성능 차이

# Personalization
- 추천 시스템이 사용자의 취향을 잘 반영하고 있는지를 평가
- 3차 필터링까지 마친 뒤 적용
- 3차 필터링에서 사용자가 추천받은 최종 10개의 메뉴 리스트 데이터 필요

In [ ]:
# 각 사용자가 추천받은 메뉴의 리스트 만들기

def make_user_list(data):
    user_list = []
    for i in range(len(data.iloc[:,0])):
      user_list.append(data.iloc[i,0])
    return user_list

In [ ]:
# ex) user 0 ~ 2 의 추천 리스트 생성
# user0_list = make_user_list(user_idx0_data)
# user1_list = make_user_list(user_idx1_data)
# user2_list = make_user_list(user_idx2_data)

In [ ]:
# 사용자들의 모든 메뉴 합치기

def make_list(user_list,total_list):
  for i in range(len(user_list)):
    total_list.append(user_list[i])
  return total_list

total_set = set(total_list) # 집합으로 변환 (중복 제거)
total_list = list(total_set) # 리스트로 변환

In [ ]:
# 각 사용자의 메뉴 리스트와 total 메뉴 리스트 비교하기

def compare_user_total(user_list,total_list):
  user_bin = []
  total_list = sorted(total_list)
  user_list = sorted(user_list)

  for j in range(len(total_list)):
    if total_list[j] in user_list :
      user_bin.append(1)
    elif total_list[j] not in user_list :
      user_bin.append(0)
  
  return user_bin

In [ ]:
# ex)
# user0_bin = compare_user_total(user0_list,total_list)
# user1_bin = compare_user_total(user1_list,total_list)
# user2_bin = compare_user_total(user2_list,total_list)

In [ ]:
# 행:user,열:menu인 행렬 만들기 (menu가 추천되었으면 1, 아니면 0)

data = [user0_bin,user1_bin,user2_bin] # 데이터로는 위에서 만든 user_bin 리스트를 넣어주기
compare_df = pd.DataFrame(data,columns=total_list)

# 행 벡터들 간의 유사도 구하기 (user들 간의 유사도 구하기)

cos_sim_df = pd.DataFrame(cosine_similarity(compare_df,compare_df))

In [ ]:
# 유사도 행렬의 대각 행렬 기준으로 평균값 구하기
# 합 = (전체 합 - user수) / 2

def mean_sim(cos_sim_df):

  total = 0
  for i in range(len(cos_sim_df)):
    col = cos_sim_df.iloc[i]
    total += sum(col)
  i = 1
  bottom = 0
  while i < len(cos_sim_df):
    bottom += i
    i += 1

  mean_sim_total = (total-len(cos_sim_df))/2
  mean_sim = mean_sim_total / bottom # 평균값

  return mean_sim

In [ ]:
# personalization 값

personalization = 1 - mean_sim
personalization